This notebook is being developed as part of the Code Kentucky Python Data Analyst pathway.<br>

Technical Specifications:
- Python 3.11.7<br>
- Instructions on setting up a Python virtual environment are contained in the README.md<br>
- Dependencies are available by running the following command: pip install -r requirements.txt

The following Code Kentucky Required Features are contained in the this notebook:<br>
- Feature #1 choice: Read multiple data files (JSON, CSV, Excel, etc.)
- Feature #2 choice: Clean the data and perform a pandas merge, then calculate some new values based on the new data set.
- Feature #3 choice: Make 3 matplotlib (or another plotting library) visualizations to display your data.
- Feature #4 choice: Utilize a Python virtual environment and include instructions in your README on how the user should set one up
- Feature #5 choice: Annotate my code with markdown cells in Jupyter Notebook, write clear code comments, and have a well-written README.md.

I've chosen the following *optional* features (if time allows):
- Feature #3 (optional): Tableau and PowerBI visualizations.
- Feature #4 (optional): Build a custom data dictionary.

---

Goal: Quantify the impact of road closures based on three metrics:
1) Total number of closures<br>
2) Frequency of closures<br>
3) Duration of closures.<br>

These are a few of the sample questions that I hope to answer:<br>
1) How many closures occur statewide each year? (Normal bar graph showing count per year?)<br>
2) How many road closures occur in each county per year? (Normal bar graph with year as x-axis and count of closures?)<br>
3) How often, or how frequently, is a single road being closed due to rainfall? (Horizonatal bar graph with roadname as Y axis or pivot table output?)<br>
4) What is the average duration of road closures?

Methodology:
1) Import/load road closure data directly from hosted web server into Pandas.<br>
2) Parse out the Latitude and Longitude by stripping unneeded hyperlink characters.<br>
3) Produce standalone latitude and longitude columns/fields, which is preferred for mapping in most BI software.<br>
3) Standardize timestamps to assist with calculating duration.<br>
4) Modify the duration calculation to show hours as float64, making it easier to use in popular BI tools.<br>
5) Summarize the results by year, county, and roadway using record counts and caculated durations.
6) If time allows, develop an overall score that takes into consideration the frequency and duration of events.

DISCLAIMER:  Results may vary.  In addition to historic data, this notebook is also utilizing current year data.  The data source is updated every 1 hour but only when there are active road closures due to weather related events.

-Chris Lambert

---

In [67]:
#Depending on the version, Pandas displays a deprecation notice about pyarrow.
#I have installed the latest pyarrow sepearetly, just in case.
#Import Python Libraries / Dependencies

import pandas as pd

2021 Dataset: Specific notes and findings about the dataset are detailed using in-line comments.

---

In [68]:
#Analyze the 2021 dataset.  I prefer to perform the import and cleaning in-memory as opposed to using local files.

#Load the 2021 dataset from the web server. 
df2021=pd.read_csv("https://storage.googleapis.com/kytc-its-2020-openrecords/toc/KYTC-TOC-Weather-Closures-Historic-2021.csv")

#Export the 2021 dataset to a csv file as a method of copying the data to the local machine.
#This gives me a backup of the file and something to reference. 
df2021.to_csv("KYTC-TOC-Weather-Closures-Historic-2021.csv", index=False)

#After reviewing the data, I noticed that the 'Route_Link' column contains a URL.
#The url needs to be cleaned to reveal the latitude and longitude, just in case I need them for mapping.
#I assume a strip that essentially performs a find/replace would work here.
df2021['Route_Link'] = df2021['Route_Link'].str.replace('https://kytc.maps.arcgis.com/apps/webappviewer/index.html?id=327a38decc8c4e5cb882dc6cd0f9d45d&zoom=14&center=', '')
df2021[['longitude','latitude']] = df2021.Route_Link.str.split(",",expand=True,)
df2021 = df2021.drop('Route_Link', axis=1)

#After reviewing the data, I noticed the following issues:
#'Reported_On' and 'End_Date' columns contain what looks like different timestamps.
#I checked with the data owners this is a known issue.  They are unable to provide additional guidance.
#I am proceeding, knowing that my duration calculation may be off by 4 hours
#I need to clean that before I can calculate the duration.
#I also noticed that the 'Comments' column contains line breaks.
df2021['End_Date'] = df2021['End_Date'].str.replace('+00:00', '')
df2021['Duration_Default'] = pd.to_datetime(df2021['End_Date']) - pd.to_datetime(df2021['Reported_On'])
df2021['Duration_Hours'] = df2021['Duration_Default'].dt.total_seconds() / 3600
df2021['Comments'] = df2021['Comments'].replace(r'[\r\n]+', ' ', regex=True) #removing line breaks from the comments column.
order=['District','County','Route','Road_Name','Begin_MP','End_MP','Comments','Reported_On','End_Date','latitude','longitude','Duration_Default','Duration_Hours']

    



#Uncomment the lines below to verify datatypes and review data in dataframe.
#print(df2021.dtypes) #verify the datatypes in the dataframe
#print(df2021.head(3)) #print the first 3 rows of the dataframe

#Export the clean 2021 dataset to a csv file to show progress.
df2021.to_csv("kytc-closures-2021-clean.csv", index=False)


2022 Dataset: Specific notes and findings about the dataset are detailed using in-line comments.

---

In [69]:
#Analyze the 2022 dataset.  I prefer to perform the import and cleaning in-memory as opposed to using local files.

#Load the 2022 dataset from the web server.
df2022=pd.read_csv("https://storage.googleapis.com/kytc-its-2020-openrecords/toc/KYTC-TOC-Weather-Closures-Historic-2022.csv")

#Export the 2022 dataset to a csv file as a method of copying the data to the local machine. 
df2022.to_csv("KYTC-TOC-Weather-Closures-Historic-2022.csv", index=False)

#After reviewing the data, I realized that the 'Route_Link' column contains a different URL from the previous dataset.
#The characters in the URL are different and the latitude and longitude are in a different order.
df2022['Route_Link'] = df2022['Route_Link'].str.replace('https://goky.ky.gov/?lat=','')
df2022['Route_Link'] = df2022['Route_Link'].str.replace('&lng=',',')
df2022['Route_Link'] = df2022['Route_Link'].str.replace('&zoom=14','')
df2022[['latitude','longitude']] = df2022.Route_Link.str.split(",",expand=True,) #removing line breaks from the comments column.
df2022 = df2022.drop('Route_Link', axis=1)

#After reviewing the data, I noticed that the 'Reported_On' and 'End_Date' columns seem to be the same timestamp.
#I need to clean that before I can calculate the duration.
#The reported_on time and the end_date are the same timestamps, which is a change from the previous dataset.
#Since it will not cause any harm to the data, I am keeping the str.replace.
#I also noticed that the 'Comments' column contains line breaks.
df2022['End_Date'] = df2022['End_Date'].str.replace('+00:00', '')
df2022['Duration_Default'] = pd.to_datetime(df2022['End_Date']) - pd.to_datetime(df2022['Reported_On'])
df2022['Duration_Hours'] = df2022['Duration_Default'].dt.total_seconds() / 3600
df2022['Comments'] = df2022['Comments'].replace(r'[\r\n]+', ' ', regex=True) #removing line breaks from the comments column.


#Uncomment the lines below to verify datatypes and review data in dataframe.
print(df2022.dtypes) #verify the datatypes in the dataframe
#print(df2022.head(3)) #print the first 3 rows of the dataframe

#Export the clean 2022 dataset to a csv file to show progress.
df2022.to_csv("kytc-closures-2022-clean.csv", index=False)


District                      int64
County                       object
Route                        object
Road_Name                    object
Begin_MP                    float64
End_MP                      float64
Comments                     object
Reported_On                  object
End_Date                     object
latitude                     object
longitude                    object
Duration_Default    timedelta64[ns]
Duration_Hours              float64
dtype: object


2023 Dataset: Specific notes and findings about the dataset are detailed using in-line comments.

---

In [70]:
#Analyze the 2023 dataset.

#The ending characters were incorrectly published and included the roadname in additon to the zoom level.

#Load the 2022 dataset from the web server. 
df2023=pd.read_csv("https://storage.googleapis.com/kytc-its-2020-openrecords/toc/KYTC-TOC-Weather-Closures-Historic-2023.csv")

#Export the 2022 dataset to a csv file as a method of copying the data to the local machine. 
df2023.to_csv("KYTC-TOC-Weather-Closures-Historic-2023.csv", index=False)

#After reviewing the data, I noticed that the 'Route_Link' column contains an error in the URL.
#This will require some additional work to strip the unwanted values.
#The placement of latitude and longitude are consistent between 2022-2024.
#The error occurs at the end of the URL, where someone has inserted a route name.
#This will require a regex to remove all characters after, and including, the &.
df2023['Route_Link'] = df2023['Route_Link'].str.replace('https://goky.ky.gov/?lat=','')
df2023['Route_Link'] = df2023['Route_Link'].str.replace('&lng=',',')
df2023['Route_Link'] = df2023['Route_Link'].str.replace('&.*', '', regex=True) #Regex was needed to compensate for an output error in the 2023 data.
df2023[['latitude','longitude']] = df2023.Route_Link.str.split(",",expand=True,)
df2023 = df2023.drop('Route_Link', axis=1)

#I need to clean that before I can calculate the duration.
#The reported_on time and the end_date are the same from the previous dataset.  No changes are needed.
#I also noticed that the 'Comments' column contains line breaks.
df2023['End_Date'] = df2023['End_Date'].str.replace('+00:00', '')
df2023['Duration_Default'] = pd.to_datetime(df2023['End_Date']) - pd.to_datetime(df2023['Reported_On'])
df2023['Duration_Hours'] = df2023['Duration_Default'].dt.total_seconds() / 3600
df2023['Comments'] = df2023['Comments'].replace(r'[\r\n]+', ' ', regex=True) #removing line breaks from the comments column.



#Uncomment the lines below to verify datatypes and review data in dataframe.
print(df2023.dtypes) #verify the datatypes in the dataframe
#print(df2023.head(3)) #print the first 3 rows of the dataframe

#Export the clean 2023 dataset to a csv file to show progress.
df2023.to_csv("kytc-closures-2023-clean.csv", index=False)

District                      int64
County                       object
Route                        object
Road_Name                    object
Begin_MP                    float64
End_MP                      float64
Comments                     object
Reported_On                  object
End_Date                     object
latitude                     object
longitude                    object
Duration_Default    timedelta64[ns]
Duration_Hours              float64
dtype: object


2024 Dataset: Specific notes and findings about the dataset are detailed using in-line comments.<br>
2024 is the current year of this analysis so this dataset will update ever hour during weather events.

---

In [71]:
#Analyze the 2024 dataset.
#Since this is 2024, this dataset is updated every 1 hour during weather events.
#This will produce different calculations throughout the year.

#Load the 2024 dataset from the web server. 
df2024=pd.read_csv("https://storage.googleapis.com/kytc-its-2020-openrecords/toc/KYTC-TOC-Weather-Closures-Historic-2024.csv")

#Export the 2024 dataset to a csv file as a method of copying the data to the local machine. 
df2024.to_csv("KYTC-TOC-Weather-Closures-Historic-2024.csv", index=False)

#The url needs to be cleaned.
#The placement of latitude and longitude are consistent between 2022-2024 but the ending of the URL in 2023 forced me to use a regex.
df2024['Route_Link'] = df2024['Route_Link'].str.replace('https://goky.ky.gov/?lat=','')
df2024['Route_Link'] = df2024['Route_Link'].str.replace('&lng=',',')
df2024['Route_Link'] = df2024['Route_Link'].str.replace('&.*', '', regex=True)
df2024[['latitude','longitude']] = df2024.Route_Link.str.split(",",expand=True,)
df2024 = df2024.drop('Route_Link', axis=1)

#I need to clean that before I can calculate the duration.
#The reported_on time and the end_date are the same timestamps but I'm keeping the strip code in just in case.
#I also noticed that the 'Comments' column contains line breaks.
df2024['End_Date'] = df2024['End_Date'].str.replace('+00:00', '')
df2024['Duration_Default'] = pd.to_datetime(df2023['End_Date']) - pd.to_datetime(df2024['Reported_On'])
df2024['Duration_Hours'] = df2024['Duration_Default'].dt.total_seconds() / 3600
df2024['Comments'] = df2024['Comments'].replace(r'[\r\n]+', ' ', regex=True) #removing line breaks from the comments column.

#Uncomment the lines below to verify datatypes and review data in dataframe.
print(df2024.dtypes) #verify the datatypes in the dataframe
#print(df2024.head(3)) #print the first 3 rows of the dataframe

#Export the clean 2023 dataset to a csv file to show progress.
df2024.to_csv("kytc-closures-2024-clean.csv", index=False)

District                      int64
County                       object
Route                        object
Road_Name                    object
Begin_MP                    float64
End_MP                      float64
Comments                     object
Reported_On                  object
End_Date                     object
latitude                     object
longitude                    object
Duration_Default    timedelta64[ns]
Duration_Hours              float64
dtype: object


Merge Dataframes and Create Reporting Dataset

---

In [72]:
#Merge the following dataframes together.
#In 2021, the columns are in a slightly different order.

#I've developed a column list to force uniformity between the datasets.
col_order=['District','County','Route','Road_Name','Begin_MP','End_MP','Comments','Reported_On','End_Date','latitude','longitude','Duration_Default','Duration_Hours']
df2021=df2021[order]
df2022=df2022[order]
df2023=df2023[order]
df2024=df2024[order]

#Merge the dataframes together.
df = pd.concat([df2021, df2022, df2023, df2024])

#Export the merged dataframe to a csv file to create a master dataset
#CSV is normally my default export file format.
df.to_csv("kytc-closures-2021-2024-report_dataset.csv", index=False) 

#Export to XLSX to make it easier to email coworkers.
#Requires the openpyxl library to be installed.
df.to_excel("kytc-closures-2021-2024-report_dataset.xlsx", index=False) 

#Parquet is useful for big data and is a good format for data lakes.
df.to_parquet("kytc-closures-2021-2024-report_dataset.parquet", index=False) 


#Each data frame contains the following columns:




Develop Visualizations (matplotlib)

---

In [73]:
#quantify the total count of road closures by year

In [74]:
#quantify the total count of road closures by county

In [75]:
#quantify the count of closures by individual roadways

In [76]:
#quantify the duration of closures by year

In [77]:
#quantify the duration of closures by county

In [78]:
#quantify the duration of closures by road

BONUS:
Map the closures on a geospacial map.<br>
Possible methods include using folium or geopandas.<br>
https://python-visualization.github.io/folium/latest/getting_started.html

---

In [79]:
#BONUS:  Perform the entire analysis in PowerBI and leave the file in the repository.

Optional Feature: Check for PowerBI or Tableau files in the repo.

---